In [2]:
import pickle
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances

In [8]:
!mv data.csv data_backup.csv

In [3]:
data = pd.read_csv('~/filtered.csv')
data

,clean,industry
0,benefit board benefit board homejoin planmembe...,marketing and advertising
1,home columbia landcar environment stewardship ...,environmental services
2,region inc homeaboutservicesclientsmentoringte...,management consulting
3,home team career servic civil design case stud...,civil engineering
4,home friedman associ pa friedman associ pa cer...,accounting
...,...,...
1694400,home byron bethani irrig district menu home go...,government administration
1694401,offici onlin store ewe athlet ewe athlet skip ...,apparel & fashion
1694402,staf undercov product las vega undercov produc...,entertainment
1694403,optometr bill solut turn problem profit turn p...,hospital & health care


In [ ]:
# industry2row = {}
# for industry, group in data.groupby('industry'):
#     industry2row[industry] = list(group.index)
# industry2row
# with open('industry2row.pkl', 'wb') as f:
#     pickle.dump(industry2row, f)

In [4]:
with open('doc2cluster.pkl', 'rb') as f:
    clusters = pickle.load(f)
    data['cluster'] = clusters
data

,clean,industry,cluster
0,benefit board benefit board homejoin planmembe...,marketing and advertising,1
1,home columbia landcar environment stewardship ...,environmental services,4
2,region inc homeaboutservicesclientsmentoringte...,management consulting,0
3,home team career servic civil design case stud...,civil engineering,0
4,home friedman associ pa friedman associ pa cer...,accounting,3
...,...,...,...
1694400,home byron bethani irrig district menu home go...,government administration,4
1694401,offici onlin store ewe athlet ewe athlet skip ...,apparel & fashion,4
1694402,staf undercov product las vega undercov produc...,entertainment,4
1694403,optometr bill solut turn problem profit turn p...,hospital & health care,2


In [5]:
data.to_csv('data.csv')

In [5]:
with open('vectors.pkl', 'rb') as f:
    vectors = pickle.load(f)

vectors.shape

(1694405, 11695315)

In [13]:
vectors[[1,4,5]]

<3x11695315 sparse matrix of type '<class 'numpy.float64'>'
	with 606 stored elements in Compressed Sparse Row format>

In [6]:
industries = data.industry.unique()
clusters = data.cluster.unique()

for i in industries:
    for c in clusters:
        print(i,c)
        left = data[(data.industry == i) & (data.cluster == c)]
        if left.shape[0] > 0:
            right = data[(data.industry != i) & (data.cluster == c)]
            assert right.shape[0] > 0
            dist = pairwise_distances(vectors[left.index], vectors[right.index], metric='cosine')
        break
    break

marketing and advertising 1


In [ ]:
left

In [ ]:
right

In [ ]:
dist

In [ ]:
cosine_values = pairwise_distances(X=tfidf_industry, Y=tfidf_other, metric='cosine')
cosine_values = scipy.sparse.csr_matrix(cosine_values)
indices = cosine_values.argmax(axis=1)
for i, index in enumerate(current_industry_indices):
    related_of[index] = current_other_indices[indices[i,0]]
print(cosine_values)